In [2]:
import org.apache.spark._
import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD

import org.apache.spark._
import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD


In [32]:
val graph = GraphLoader.edgeListFile(sc,"data/soc-karate.mtx")
                .mapEdges(e => scala.util.Random.nextFloat)
                .mapVertices((vid,data) =>  false)
graph.vertices.collect

graph: org.apache.spark.graphx.Graph[Boolean,Float] = org.apache.spark.graphx.impl.GraphImpl@1ca08b31
res20: Array[(org.apache.spark.graphx.VertexId, Boolean)] = Array((34,false), (4,false), (16,false), (22,false), (28,false), (30,false), (14,false), (32,false), (24,false), (6,false), (8,false), (12,false), (18,false), (20,false), (26,false), (10,false), (2,false), (13,false), (19,false), (15,false), (21,false), (25,false), (29,false), (11,false), (27,false), (33,false), (23,false), (1,false), (17,false), (3,false), (7,false), (9,false), (31,false), (5,false))


In [3]:
def independant_cascade(graph:Graph[String,Float]):Graph[String,Float]={
    def mergeMsg (m1:String, m2:String):String = {
        if(m1=="become_inactive" || m2 == "become_inactive"){return "become_inactive"}
        if (m1 == "become_infected" || m2 == "become_infected"){return "become_infected"}
        return "nothing"
    }

    def vprog(VertexId:VertexId, VD:String, A:String): String ={
        if (VD == "susceptible" && A == "become_infected"){ return "infected"}
        if(VD=="infected" && A == "become_inactive"){return "inactive"}
        return VD
    }

    def sendMsg(triplet:EdgeTriplet[String, Float]): Iterator[(VertexId, String)]={
        if (triplet.srcAttr == "infected"){
            if (scala.util.Random.nextFloat < triplet.attr){
                return Iterator((triplet.dstId,"become_infected"),(triplet.srcId,"become_inactive"))
            }
        return Iterator((triplet.srcId,"become_inactive"))
        }
        return Iterator()
    }
    
    return graph.pregel("nothing",Int.MaxValue,EdgeDirection.Out)(vprog,sendMsg,mergeMsg)
    .mapVertices((vid,vdata) => if(vdata == "infected") "inactive" else vdata)
}

independant_cascade: (graph: org.apache.spark.graphx.Graph[String,Float])org.apache.spark.graphx.Graph[String,Float]


In [26]:
independant_cascade(graph).vertices.collect

<console>: 51: error: type mismatch;

In [4]:
type VD = (String,Float,Float)
type ED = Float
type Message = (String,Float)

def linear_threshhold(graph:Graph[VD,ED]):Graph[VD,ED]={
    def mergeMsg (m1:Message, m2:Message):Message = {
        if(m1._1=="become_inactive" || m2._1 == "become_inactive"){return ("become_inactive",0f)}
        if (m1._1 == "become_infected" && m2._1 == "become_infected"){return ("become_infected",m1._2+m2._2)}
        if (m1._1 == "become_infected" ){return m1}
        if (m2._1 == "become_infected" ){return m2}
        return ("nothing",0f)
    }

    def vprog(VertexId:VertexId, vdata:VD, m:Message): VD ={
        if (m._1 == "become_inactive"){
            return ("inactive", vdata._2, vdata._3)
        }
        if (m._1 == "become_infected"){
            val state = vdata._1
            val new_infection_rate = vdata._2 + m._2
            val threshold = vdata._3
            if (state == "susceptible" && new_infection_rate > threshold){
                return ("infected",new_infection_rate,threshold)
            }
            return (state,new_infection_rate,threshold)
        }
        return vdata
    }

    def sendMsg(triplet:EdgeTriplet[VD, ED]): Iterator[(VertexId, Message)]={
        if (triplet.srcAttr._1 == "infected"){
            return Iterator((triplet.dstId,("become_infected",triplet.attr)),(triplet.srcId,("become_inactive",0f)))

        return Iterator((triplet.srcId,("become_inactive",0f)))
        }
        return Iterator()
    }
    
    return graph.pregel(("nothing",0f),Int.MaxValue,EdgeDirection.Out)(vprog,sendMsg,mergeMsg)
}

defined type alias VD
defined type alias ED
defined type alias Message
linear_threshhold: (graph: org.apache.spark.graphx.Graph[VD,ED])org.apache.spark.graphx.Graph[VD,ED]


In [16]:
def normalize_graph_weights[VD](graph:Graph[VD,Float])(implicit m: ClassManifest[VD]):Graph[VD,Float] = {
    val weight_sums_per_dst = graph.triplets.map(t => (t.dstId,t.attr))
                            .reduceByKey( _ + _)
    val newEdges = graph.edges.map(e => (e.dstId,e))
         .join(weight_sums_per_dst).map{ case ((dstId,(edge,weightSum)))=>Edge(edge.srcId,edge.dstId,edge.attr/weightSum)}
    return Graph(graph.vertices,newEdges)
}

val graph = GraphLoader.edgeListFile(sc,"data/soc-karate.mtx")
                .mapEdges(e => scala.util.Random.nextFloat)
                .mapVertices[VD]((vid,i) =>  if(scala.util.Random.nextFloat <0.1) ("infected",1f, scala.util.Random.nextFloat) else ("susceptible",0f, scala.util.Random.nextFloat))
graph.edges.collect

normalize_graph_weights: [VD](graph: org.apache.spark.graphx.Graph[VD,Float])(implicit m: ClassManifest[VD])org.apache.spark.graphx.Graph[VD,Float]
graph: org.apache.spark.graphx.Graph[VD,Float] = org.apache.spark.graphx.impl.GraphImpl@6f5cf07d
res11: Array[org.apache.spark.graphx.Edge[Float]] = Array(Edge(2,1,0.7104812), Edge(3,1,0.6240713), Edge(3,2,0.13820559), Edge(4,1,0.3864687), Edge(4,2,0.46350425), Edge(4,3,0.16832858), Edge(5,1,0.52650243), Edge(6,1,0.6943162), Edge(7,1,0.8280435), Edge(7,5,0.81049347), Edge(7,6,0.056300104), Edge(8,1,0.29778677), Edge(8,2,0.60662943), Edge(8,3,0.6356645), Edge(8,4,0.1484586), Edge(9,1,0.80549085), Edge(9,3,0.5572843), Edge(10,3,0.4350739), Edge(11,1,0.85606533), Edge(11,5,0.2947697), Edge(11,6,0.8379629), Edge(12,1,0.5636415), Edge(13,1,0.26...


In [28]:
normalize_graph_weights(graph).edges.collect

res17: Array[org.apache.spark.graphx.Edge[Float]] = Array(Edge(3,2,0.002006809), Edge(4,2,0.088723645), Edge(7,6,0.26730868), Edge(8,2,0.29874447), Edge(8,4,0.4271791), Edge(11,6,0.100956455), Edge(13,4,0.15480156), Edge(14,2,0.15437755), Edge(14,4,0.4180193), Edge(17,6,0.63173485), Edge(18,2,0.28450096), Edge(20,2,0.013835116), Edge(22,2,0.07062683), Edge(26,24,0.24121854), Edge(28,24,0.06453526), Edge(30,24,0.03656913), Edge(31,2,0.087184705), Edge(32,26,1.0), Edge(33,16,0.5309698), Edge(33,24,0.33191332), Edge(33,30,0.508525), Edge(33,32,0.66386473), Edge(34,10,1.0), Edge(34,14,1.0), Edge(34,16,0.4690302), Edge(34,20,1.0), Edge(34,24,0.32576373), Edge(34,28,1.0), Edge(34,30,0.491475), Edge(34,32,0.33613527), Edge(34,34,1.0), Edge(2,1,0.07558371), Edge(3,1,0.054871283), Edge(4,1,0.043...


In [29]:
linear_threshhold(graph).vertices.filter(vdata => vdata._2._2 >0).collect

res18: Array[(org.apache.spark.graphx.VertexId, VD)] = Array((2,(susceptible,0.24847716,0.7130439)), (9,(susceptible,0.08883333,0.45760077)), (31,(inactive,1.0,0.356121)))


In [8]:
def selectVerticeWithMaxDegree(graph:Graph[Boolean,Float]):Graph[Boolean,Float] = {
    val maxDegreeVertice = graph.triplets.filter(triplet => !triplet.srcAttr && !triplet.dstAttr)
                             .map(triplet => (triplet.srcId,1))
                             .reduceByKey(_+_)
                             .reduce((v1,v2) => if(v1._2 >= v2._2 ) v1 else v2)._1
    return graph.mapVertices((vid, vdata) => vid == maxDegreeVertice || vdata)
}

val selectKVerticesWithMaxDegrees = (k:Int) => Function.chain(List.fill(k)(selectVerticeWithMaxDegree _ ))

selectVerticeWithMaxDegree: (graph: org.apache.spark.graphx.Graph[Boolean,Float])org.apache.spark.graphx.Graph[Boolean,Float]
selectKVerticesWithMaxDegrees: Int => (org.apache.spark.graphx.Graph[Boolean,Float] => org.apache.spark.graphx.Graph[Boolean,Float]) = $Lambda$2987/0x0000000101000840@101724b4


In [ ]:
def selectVerticeRandom(graph:Graph[Boolean,Float]):Graph[Boolean,Float] = {
    graph.
}

In [33]:
selectKVerticesWithMaxDegrees (10) (graph).vertices.join(graph.outDegrees).collect

res21: Array[(org.apache.spark.graphx.VertexId, (Boolean, Int))] = Array((34,(true,18)), (4,(true,3)), (22,(true,2)), (28,(true,3)), (30,(false,2)), (14,(true,4)), (32,(true,4)), (6,(false,1)), (8,(true,4)), (12,(false,1)), (18,(false,2)), (20,(false,2)), (26,(false,2)), (10,(false,1)), (2,(false,1)), (13,(false,2)), (29,(false,1)), (11,(true,3)), (33,(true,11)), (17,(false,2)), (3,(false,2)), (7,(true,3)), (9,(false,2)), (31,(false,2)), (5,(false,1)))


In [58]:
graph.mapVertices((vid,selected) =>  if (selected) "infected" else "susceptible")

res39: org.apache.spark.graphx.Graph[String,Float] = org.apache.spark.graphx.impl.GraphImpl@4ea135d7


In [93]:
val graph = GraphLoader.edgeListFile(sc,"data/soc-karate.mtx")
                .mapEdges(e => scala.util.Random.nextFloat)
                .mapVertices((vid,data) =>  false)

val graph_init = selectKVerticesWithMaxDegrees (10) (graph).mapVertices(
                    (vid,selected) =>  if (selected) "infected" else "susceptible")

val graph_result = independant_cascade(graph_init)

graph_result.vertices.map(v => if (v._2 == "susceptible") 0 else 1).reduce((a,b) => a+b)

graph: org.apache.spark.graphx.Graph[Boolean,Float] = org.apache.spark.graphx.impl.GraphImpl@188a3dce
graph_init: org.apache.spark.graphx.Graph[String,Float] = org.apache.spark.graphx.impl.GraphImpl@1d5226cf
graph_result: org.apache.spark.graphx.Graph[String,Float] = org.apache.spark.graphx.impl.GraphImpl@552a8f49
res68: Int = 22


In [ ]:
val graph = GraphLoader.edgeListFile(sc,"data/soc-karate.mtx")
                .mapEdges(e => scala.util.Random.nextFloat)
                .mapVertices((vid,data) =>  false)

val graph_init = selectKVerticesWithMaxDegrees (10) (graph).mapVertices(
                    (vid,selected) =>  if (selected) "infected" else "susceptible")

val graph_result = independant_cascade(graph_init)

graph_result.vertices.map(v => if (v._2 == "susceptible") 0 else 1).reduce((a,b) => a+b)

In [6]:
val edges =  sc.textFile("data/soc-sign-bitcoinotc.csv")
                .map(f => f.split(","))
                .map(a => Edge(a(0).toLong, a(1).toLong, a(2).toLong / 10f))
                .filter(e => (e.attr > 0))

val vertices = edges.map(e => (e.srcId, false)).distinct()

val graph = Graph(vertices,edges)

val graph_init = selectKVerticesWithMaxDegrees (1000) (graph).mapVertices(
                    (vid,selected) =>  if (selected) "infected" else "susceptible")

val graph_result = independant_cascade(graph_init)

graph_result.vertices.map(v => if (v._2 == "susceptible") 0 else 1).reduce((a,b) => a+b)

<console>: 46: error: not found: value selectKVerticesWithMaxDegrees

In [60]:
val edges =  sc.textFile("data/soc-sign-bitcoinotc.csv")
                .map(f => f.split(","))
                .map(a => Edge(a(0).toLong, a(1).toLong, a(2).toLong / 10f))
                .filter(e => (e.attr > 0))

val vertices = edges.map(e => (e.srcId, false)).distinct()

val graph = Graph(vertices,edges)

val graph_init = selectKVerticesWithMaxDegrees (100) (graph).mapVertices(
                    (vid,selected) =>  if (selected) ("infected",1f, randomRange(0.4f,1f)) else ("susceptible",0f, randomRange(0.4f,1f)))

val graph_result = linear_threshhold(normalize_graph_weights(graph_init))

graph_result.vertices.map(v => if (v._2._1== "susceptible") 0 else 1).reduce((a,b) => a+b)

edges: org.apache.spark.rdd.RDD[org.apache.spark.graphx.Edge[Float]] = MapPartitionsRDD[8768] at filter at <console>:49
vertices: org.apache.spark.rdd.RDD[(org.apache.spark.graphx.VertexId, Boolean)] = MapPartitionsRDD[8772] at distinct at <console>:51
graph: org.apache.spark.graphx.Graph[Boolean,Float] = org.apache.spark.graphx.impl.GraphImpl@174b25cf
graph_init: org.apache.spark.graphx.Graph[(String, Float, Float),Float] = org.apache.spark.graphx.impl.GraphImpl@1b616ef8
graph_result: org.apache.spark.graphx.Graph[VD,ED] = org.apache.spark.graphx.impl.GraphImpl@5370da27
res46: Int = 2287
